In [1]:
import sys
sys.path.append("../")

In [2]:
from utils.chroma_store import ChromaStore
from chromadb.utils import embedding_functions
from transformers import pipeline
import pandas as pd
import time
import requests
import re
import json
import uuid
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option('max_colwidth', 800)

Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [3]:
### Parameters
data_version = "data/third_version" # We use local version of latest data since remote dvc only contains data/second_version
COLLECTION_NAME_MAP = {"mind_data": "Mind", "nhs_data": "NHS"}
DEFAULT_EMBED_INSTRUCTION = "Represent the document for retrieval: "
DEFAULT_QUERY_INSTRUCTION = (
    "Represent the question for retrieving supporting documents: "
)

CHROMA_ENDPOINT = "localhost"
chroma_client = ChromaStore(chroma_server_hostname=CHROMA_ENDPOINT, chroma_server_port=8000)
OPENLLM_ENDPOINT = "http://localhost:3000/v1/generate"

MIND_CHUNK_SIZE = 1000
MIND_CHUNK_OVERLAP = 100
NHS_CHUNK_SIZE = 2000
NHS_CHUNK_OVERLAP = 50

EMBED_MODEL = "hkunlp/instructor-base"
EMBED_FUNC = embedding_functions.InstructorEmbeddingFunction(model_name=EMBED_MODEL, instruction=DEFAULT_QUERY_INSTRUCTION)
N_CLOSEST_MATCHES = 3

QUESTIONS = [
"Can you tell me about depression?",
"What lifestyle changes are suggested for depression?",
"Are there different types of depressions?"
]

TEMPLATES = {
    "simple": "Context: {context}\n\nQuestion: {question}\n\n",
    "complex": """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Answer:""",
    "advanced": """You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
{context}
Question: {question}
Answer:""",
}
MAX_OUTPUT_LENGTH = 300
TEMPERATURE = 0.1

Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Load pretrained SentenceTransformer: hkunlp/instructor-base
load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
def use_local_base_model(template, messages):
    context = messages["context"]
    input_text = template.format(question=messages["prompt_query"], context=context)
    print(f"Prompt to LLM : {input_text}")
    llm = pipeline(model="google/flan-t5-base", task="text2text-generation")
    st = time.perf_counter()
    data = llm(input_text, max_length=MAX_OUTPUT_LENGTH, temperature=TEMPERATURE)
    time_taken = time.perf_counter()-st
    print(f"Time taken to generate response: {time_taken} sec")
    return time_taken, data[0]["generated_text"]

def query_vector_store(chroma_client, query_text, collection_name, n_results, embedding_function):
    result_dict = chroma_client.query_collection(
        collection_name=collection_name,
        query_texts=query_text,
        n_results=n_results,
        embedding_function=embedding_function,
    )
    documents = " ".join(result_dict["documents"][0])
    return documents


def clean_fastchat_t5_output(answer):
    answer = re.sub(r"<pad>\s+", "", answer)
    answer = re.sub(r"  ", " ", answer)
    answer = re.sub(r"\n$", "", answer)
    return answer

def query_llm(template, messages):
    context = messages["context"]
    input_text = template.format(question=messages["prompt_query"], context=context)
    print(f"Prompt to LLM : {input_text}")
    payload = {"prompt": str(input_text), "llm_config": {"temperature": TEMPERATURE, "max_new_tokens": MAX_OUTPUT_LENGTH}}
    st = time.perf_counter()
    response = requests.post(url=OPENLLM_ENDPOINT,
                             data=json.dumps(payload),
                             headers={"Content-Type": "application/json"}
                            )
    time_taken = time.perf_counter()-st
    print(f"Time taken to generate response: {time_taken} sec")
    summary_txt = str(json.loads(response.text)["responses"][0])
    # summary_txt = clean_fastchat_t5_output(summary_txt)         # ONLY REQUIRED for Fastchat model
    return time_taken, summary_txt


def run_experiment(questions, template, is_baseline = False):
    mind_responses, nhs_responses, mind_time, nhs_time = [], [], [], []
    for collection, source in COLLECTION_NAME_MAP.items():
        for prompt in questions:
            # Query vector store
            context = query_vector_store(
                chroma_client=chroma_client,
                query_text=prompt,
                collection_name=collection,
                n_results=N_CLOSEST_MATCHES,
                embedding_function=EMBED_FUNC,
            )
            
            # Create a dict of prompt and context
            message = {"prompt_query": prompt, "context": context}
            
            # Query LLM by passing query and context
            if is_baseline:
                time_taken, assistant_response = use_local_base_model(template=template, messages=message)
            else:
                time_taken, assistant_response = query_llm(template=template, messages=message)
            
            print(f"Question: {prompt}")
            print(f"LLM Response: {assistant_response}")
            if source == "Mind":
                mind_responses.append(assistant_response)
                mind_time.append(round(time_taken, 2))
            else:
                nhs_responses.append(assistant_response)
                nhs_time.append(round(time_taken, 2))

            print("="*80)

    result_df = pd.DataFrame()
    result_df['questions'] = questions
    result_df['mind_responses'] = mind_responses
    result_df['nhs_responses'] = nhs_responses
    result_df['mind_time'] = mind_time
    result_df['nhs_time'] = nhs_time
    return result_df

# Experiments

> Note: This requires a local chroma store running in the background. To run a chroma store server, run the following steps

```bash
docker pull ghcr.io/chroma-core/chroma:0.4.3
docker run -it -p 8000:8000 ghcr.io/chroma-core/chroma:0.4.3
```

This will start chroma server on localhost at port 8000.

Run the embedding pipeline in a separate terminal to store the embeddings for the latest version of dataset.

```bash
python run.py --embed
```

> Note: Inside `data_embedding_pipeline.py`, comment out the `compute_embedding_drift` step as its not required.

### Flan-T5 Base Baseline

In [12]:
result_df = run_experiment(QUESTIONS, TEMPLATES['simple'], is_baseline=True)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own. But it might be depression if the feelings be

Token indices sequence length is longer than the specified maximum sequence length for this model (697 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 4.428688575000706 sec
Question: Can you tell me about depression?
LLM Response: Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings
Prompt to LLM : Context: make ourselves feel better or distract ourselves. But they can make us feel worse in the long term.  Sleep, diet and exercise. Some of us may struggle with sleeping, getting active, or keeping a healthy diet. And if we find these things difficult, it can affect our mood. These things are unlikely to cause depression on their own. But they could make us more vulnerable to it. for other activities you enjoy, su

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 0.7050035969987221 sec
Question: What lifestyle changes are suggested for depression?
LLM Response: Sleep, diet and exercise
Prompt to LLM : Context: a reason and pass on their own. But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.  I learnt to cope by telling myself I don't care. Telling myself emphatically that nothing was important, I just don’t care, was my way of coping Read David's story Are there different types of depression? If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're likely to be offered. You may find that the severity of your depression changes over time. Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too: Persistent depressive disorder (PDD). PD

...

Time taken to generate response: 3.7173366280003393 sec
Question: Can you tell me about depression?
LLM Response: Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.
Prompt to LLM : Context: Living with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicine. It's very important to take your antidepressants as prescribed, even if you start to feel better. If you stop taking them too soon, your depression could return. Stopping suddenly can also increase the risk of withdrawal side effects. Talk to a doctor or pharmacist if you have any questions or concerns about the medicine you're taking. The leaflet that comes with your medicine will have information about possible interactions with other medicines or supplements. Check with a doct

Token indices sequence length is longer than the specified maximum sequence length for this model (1226 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 2.3285896579982364 sec
Question: What lifestyle changes are suggested for depression?
LLM Response: Get more exercise, cutting down on alcohol, giving up smoking and eating healthily
Prompt to LLM : Context: are. how long depression lasts. the impact on your daily life. A few people with severe depression may have symptoms of psychosis . You'll usually be asked to answer a set of questions to help doctors assess how severe your condition is. Grief and depression. It can be difficult to distinguish between grief and depression. They share many of the same characteristics, but there are important differences between them. Grief is an entirely natural response to a loss, while depression is an illness. People who are grieving may have symptoms of depression, but they usually improve with time. For some people, bereavement can lead to depression. If you're grieving it's normal to have feelings of sadness. Some other symptoms of depression are rare with grie

Token indices sequence length is longer than the specified maximum sequence length for this model (1326 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 4.952952362997166 sec
Question: Are there different types of depressions?
LLM Response: There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines.


In [13]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings","Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",4.43,3.72
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise","Get more exercise, cutting down on alcohol, giving up smoking and eating healthily",0.71,2.33
2,Are there different types of depressions?,"There are many different mental health problems. Some of them have similar symptoms, so you may experience the symptoms of more than one mental health problem, or be given several diagnoses at once. Or you might not have any particular diagnosis, but still be finding things very difficult.","There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines.",3.15,4.95


In [14]:
result_df = run_experiment(QUESTIONS, TEMPLATES['complex'], is_baseline=True)

Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb

Token indices sequence length is longer than the specified maximum sequence length for this model (750 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 1.6155284410015156 sec
Question: Can you tell me about depression?
LLM Response: It can cause a range of other changes to how you feel or behave.
Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
make ourselves feel better or distract ourselves. But they can make us feel worse in the long term.  Sleep, diet and exercise. Some of us may struggle with sleeping, getting active, or keeping a healthy diet. And if we find these things difficult, it can affect our mood. These things are unlikely to cause depression on their own. But they could make us more vulnerable to it. for other activities you enjoy, such as: Gaming. Reading. Watching films. For more ideas, see our pages on arts and creative therapies , nature and mental health , sleep problems , stress , re

Token indices sequence length is longer than the specified maximum sequence length for this model (567 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 0.6919020750028722 sec
Question: What lifestyle changes are suggested for depression?
LLM Response: Sleep, diet and exercise
Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
a reason and pass on their own. But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.  I learnt to cope by telling myself I don't care. Telling myself emphatically that nothing was important, I just don’t care, was my way of coping Read David's story Are there different types of depression? If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're likely to be offered. You may

...

Time taken to generate response: 6.663905643999897 sec
Question: Can you tell me about depression?
LLM Response: It's more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition with real symptoms. Depression is not a sign of weakness or something you can "snap out of" by "pulling yourself together".
Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Living with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicine. It's very important to take your antidepressants as prescribed, even 

Token indices sequence length is longer than the specified maximum sequence length for this model (1279 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 3.2322566250004456 sec
Question: What lifestyle changes are suggested for depression?
LLM Response: getting more exercise, cutting down on alcohol, giving up smoking and eating healthily
Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
are. how long depression lasts. the impact on your daily life. A few people with severe depression may have symptoms of psychosis . You'll usually be asked to answer a set of questions to help doctors assess how severe your condition is. Grief and depression. It can be difficult to distinguish between grief and depression. They share many of the same characteristics, but there are important differences between them. Grief is an entirely natural response to a loss, while depression is an illness. People who are grieving may 

Token indices sequence length is longer than the specified maximum sequence length for this model (1379 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 2.1179904470009205 sec
Question: Are there different types of depressions?
LLM Response: Yes


In [15]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,It can cause a range of other changes to how you feel or behave.,"It's more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition with real symptoms. Depression is not a sign of weakness or something you can ""snap out of"" by ""pulling yourself together"".",1.62,6.66
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise","getting more exercise, cutting down on alcohol, giving up smoking and eating healthily",0.69,3.23
2,Are there different types of depressions?,Yes,Yes,0.78,2.12


In [16]:
result_df = run_experiment(QUESTIONS, TEMPLATES['advanced'], is_baseline=True)

Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and les

Token indices sequence length is longer than the specified maximum sequence length for this model (796 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 3.4709833179986163 sec
Question: Can you tell me about depression?
LLM Response: Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. This page covers: Common signs and symptoms of depression.
Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
make ourselves feel better or distract ourselves. But they can make us feel worse in the lon

Token indices sequence length is longer than the specified maximum sequence length for this model (613 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 0.8053261169989128 sec
Question: What lifestyle changes are suggested for depression?
LLM Response: Sleep, diet and exercise.
Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
a reason and pass on their own. But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months.  I learnt to cope by telling myself I don't care. Telling myself emphatically that nothing was important, I just don’t care, was my way of coping Read Da

...

Time taken to generate response: 4.697112489000574 sec
Question: Can you tell me about depression?
LLM Response: Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.
Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
Living with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicin

Token indices sequence length is longer than the specified maximum sequence length for this model (1325 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 2.9586929610013613 sec
Question: What lifestyle changes are suggested for depression?
LLM Response: Get more exercise, cut down on alcohol, give up smoking and eating healthily.
Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
are. how long depression lasts. the impact on your daily life. A few people with severe depression may have symptoms of psychosis . You'll usually be asked to answer a set of questions to help doctors assess how severe your condition is. Grief and depression. It can be diffic

Token indices sequence length is longer than the specified maximum sequence length for this model (1425 > 512). Running this sequence through the model will result in indexing errors


Time taken to generate response: 3.4404237600028864 sec
Question: Are there different types of depressions?
LLM Response: There are different types of depression, and some conditions where depression may be one of the symptoms. These include:


In [17]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. This page covers: Common signs and symptoms of depression.,"Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",3.47,4.70
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise.","Get more exercise, cut down on alcohol, give up smoking and eating healthily.",0.81,2.96
2,Are there different types of depressions?,There are many different types of depression.,"There are different types of depression, and some conditions where depression may be one of the symptoms. These include:",1.11,3.44


### Flan-T5 Large

In [25]:
result_df = run_experiment(QUESTIONS, TEMPLATES['simple'], is_baseline=False)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own. But it might be depression if the feelings be

In [26]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things.,Depression is more than simply feeling unhappy or fed up for a few days.,2.50,1.37
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise","getting more exercise, cutting down on alcohol, giving up smoking and eating healthily",0.77,1.30
2,Are there different types of depressions?,"If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’.","There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines.",1.26,2.68


In [27]:
result_df = run_experiment(QUESTIONS, TEMPLATES['complex'], is_baseline=False)

Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb

In [28]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things.,Yes,1.19,1.01
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise.","Exercise, cutting down on alcohol, giving up smoking and eating healthily.",0.76,1.31
2,Are there different types of depressions?,Yes.,Yes,0.74,1.02


In [29]:
result_df = run_experiment(QUESTIONS, TEMPLATES['advanced'], is_baseline=False)

Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and les

In [30]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave.,Depression is a real illness with real symptoms.,1.60,1.28
1,What lifestyle changes are suggested for depression?,Getting active and eating well. Talking therapies.,"Many people with depression benefit by making lifestyle changes, such as getting more exercise, cutting down on alcohol, giving up smoking and eating healthily.",0.89,1.67
2,Are there different types of depressions?,"If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're likely to be offered. Sometimes you might hear depression being called ‘major depressive disorder’. There are some other types of depression too: Persistent depressive disorder (PDD). PDD is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression. Seasonal affective disorder (SAD). SAD is depression that","There are different types of depression, and some conditions where depression may be one of the symptoms. These include: postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines. bipolar disorder – also known as ""manic depression"", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex. seasonal affective disorder (SAD) – also known as ""winter depression"", SAD is a type of depressi...",3.53,6.11


### Flan-T5 XL

In [31]:
result_df = run_experiment(QUESTIONS, TEMPLATES['simple'], is_baseline=False)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own. But it might be depression if the feelings be

In [32]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,How it affects your life,"Answer: - Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",5.07,3.23
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise","getting more exercise, cutting down on alcohol, giving up smoking and eating healthily",1.12,2.17
2,Are there different types of depressions?,"If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’.","There are different types of depression, and some conditions where depression may be one of the symptoms.",1.72,2.36


In [33]:
result_df = run_experiment(QUESTIONS, TEMPLATES['complex'], is_baseline=False)

Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb

In [34]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,It is a mental health problem that involves having a low mood or losing interest and enjoyment in things.,It is a real illness with real symptoms.,1.66,2.14
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise","Getting more exercise, cutting down on alcohol, giving up smoking and eating healthily.",1.10,2.19
2,Are there different types of depressions?,Yes.,Yes.,1.16,1.96


In [35]:
result_df = run_experiment(QUESTIONS, TEMPLATES['advanced'], is_baseline=False)

Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and les

In [36]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things.,"Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days.",1.76,3.30
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise.","Getting more exercise, cutting down on alcohol, giving up smoking and eating healthily.",1.13,2.35
2,Are there different types of depressions?,There are some other types of depression too: Persistent depressive disorder (PDD) is continuous depression that lasts for two years or more. You may also hear it called dysthymia or chronic depression. Seasonal affective disorder (SAD) is depression that,"There are different types of depression, and some conditions where depression may be one of the symptoms.",2.67,2.58


### FastChat 3B

Huggigface: https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

In [18]:
result_df = run_experiment(QUESTIONS, TEMPLATES['simple'], is_baseline=False)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own. But it might be depression if the feelings be

In [19]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"Yes, depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you may find day-to-day life much more difficult. You may also experience suicidal feelings\n. It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty.\n When","Yes, depression is a mental health condition that affects people in different ways and can cause a wide variety of symptoms. The symptoms of depression range from mild to severe and can include psychological, physical, and social symptoms. Depression can be caused by life-changing events such as bereavement, losing a job, or giving birth. The symptoms of depression can be complex and vary widely between people. The symptoms of depression can be described as mild, moderate, or severe based on the severity of the symptoms, the duration of the symptoms, and the impact on daily life. There are different types of depression and conditions where depression may be one of the symptoms. It is important to seek help from a doctor if you think you may be depressed and to seek treatment as soon as...",9.34,11.48
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise.",Answer: getting more exercise,1.58,3.51
2,Are there different types of depressions?,"Yes, there are different types of depression. Some people with depression may have a milder form of the condition, called 'minor depression'. This is depression that lasts for a few weeks or months, and doesn't interfere with daily life. Other people with depression may have a more severe form of the condition, called 'major depression'. There are also some other types of depression too:\n Persistent depressive disorder (PDD)","Yes, there are different types of depression. These include:\n postnatal depression – sometimes new mothers, fathers or partners develop depression after they have a baby; this is known as postnatal depression and it's treated in a similar way to other types of depression, with talking therapies and antidepressant medicines.\n bipolar disorder – also known as ""manic depression"", in bipolar disorder there are spells of both depression and excessively high mood (mania); the depression symptoms are similar to depression, but the bouts of mania can include harmful behaviour, such as gambling, going on spending sprees and having unsafe sex.\n seasonal affective disorder (SAD) – also known as ""winter depression"", SAD is a type of depression with a seasonal pattern usually related to winter.\...",5.90,11.68


In [20]:
result_df = run_experiment(QUESTIONS, TEMPLATES['complex'], is_baseline=False)

Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb

In [21]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"Yes, depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you may find day-to-day life much more difficult. You may also experience suicidal feelings.","Yes, depression is a real illness with real symptoms. It can cause lasting feelings of unhappiness and hopelessness, as well as physical symptoms such as feeling constantly tired, sleeping badly, and having no appetite or sex drive. The symptoms of depression range from mild to severe and can be caused by life-changing events such as bereavement, losing a job, or giving birth. The symptoms of depression can be complex and vary widely between people. It is important to seek help from a doctor if you think you may be depressed. The causes of depression can be life-changing events such as bereavement, loss of a job, or giving birth. The symptoms of depression can be complex and vary widely between people. The severity of depression can be described by doctors based on",7.03,11.81
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise.","Getting more exercise, cutting down on alcohol, giving up smoking and eating healthily.",1.66,4.24
2,Are there different types of depressions?,"Yes, there are different types of depression. Some examples include major depressive disorder, persistent depressive disorder (PDD), seasonal affective disorder (SAD), and mixed episodes. The severity of depression can change over time and can be described by different terms.","Yes, there are different types of depression, including postnatal depression, bipolar disorder, seasonal affective disorder (SAD), and premenstrual dysphoric disorder (PMDD). These conditions can have depression as one of the symptoms.",4.02,5.78


In [22]:
result_df = run_experiment(QUESTIONS, TEMPLATES['advanced'], is_baseline=False)

Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and les

In [23]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you may find day-to-day life much more difficult. You may also experience suicidal feelings. It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. Depression can be a symptom of other mental health problems and can affect your day-to-day life. Common signs and symptoms","Yes, depression is a serious mental health condition that can affect adults. It is more than simply feeling unhappy or fed up for a few days. Depression can cause a wide variety of symptoms, including lasting feelings of unhappiness and hopelessness, losing interest in the things you used to enjoy, and feeling very tearful. The symptoms of depression range from mild to severe, and can be caused by life-changing events such as bereavement, losing a job, or giving birth. The symptoms of depression can be complex and vary widely between people. The severity of depression can be described based on the symptoms, how often you get symptoms and how bad they are, how long depression lasts, the impact on your daily life, and the severity of the symptoms. There are different types of",9.65,12.16
1,What lifestyle changes are suggested for depression?,"Sleep, diet and exercise are suggested for depression. Engaging in activities that you enjoy, maintaining a healthy diet, and trying some physical activity are also suggested. Talking therapies are also recommended to treat depression.","The suggested lifestyle changes for depression include getting more exercise, cutting down on alcohol, giving up smoking, and eating healthily. These changes can help lift your mood, reduce stress and anxiety, encourage the release of endorphins (your body's feel-good chemicals), and improve self-esteem. They can also improve your general health and general well-being. It is important to take your medicine as prescribed, check with a doctor first if you plan to take any over-the-counter remedies such as painkillers, or any nutritional supplements. These can sometimes interfere with antidepressants.",3.49,9.49
2,Are there different types of depressions?,"Yes, there are different types of depression. There is major depressive disorder, persistent depressive disorder (PDD), seasonal affective disorder (SAD), and mixed episodes. The severity of depression can change over time and there are different treatments available. Some people may experience the symptoms of more than one mental health problem, or be given several diagnoses at once. The symptoms of depression can vary in severity and can change over time. It is important to seek help if you are experiencing symptoms of depression and to understand that everyone's experience is different.","Yes, there are different types of depression, including postnatal depression, bipolar disorder, seasonal affective disorder (SAD), and premenstrual dysphoric disorder (PMDD). These conditions can have symptoms of depression as one of their symptoms. The severity of depression can vary from mild to severe, and the symptoms can range from mild to severe. It's important to seek help from a doctor if you think you may have depression.",7.14,8.35


### OpenLLama 3B

Huggingface : https://huggingface.co/openlm-research/open_llama_3b_v2

> Note: Since this is a base moddel and not a Instruct-tuned model, all it does is completes the input prompt. Not usefult at all.

In [13]:
result_df = run_experiment(QUESTIONS, TEMPLATES['simple'], is_baseline=False)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own. But it might be depression if the feelings be

In [14]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings. It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood become d...","Context: Overview - Depression in adults. Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can ""snap out of"" by ""pulling yourself together"". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people. How to tell if you have depression. Depression affects people in different ways and ...",10.39,11.45
1,What lifestyle changes are suggested for depression?,"Context: make ourselves feel better or distract ourselves. But they can make us feel worse in the long term. Sleep, diet and exercise. Some of us may struggle with sleeping, getting active, or keeping a healthy diet. And if we find these things difficult, it can affect our mood. These things are unlikely to cause depression on their own. But they could make us more vulnerable to it. for other activities you enjoy, such as: Gaming. Reading. Watching films. For more ideas, see our pages on arts and creative therapies, nature and mental health, sleep problems, stress, relaxation and mindfulness. Think about your diet. It can be difficult to maintain a healthy diet when we’re struggling with our mental health. But eating regularly can keep your blood sugar stable and really make a differe...","Context: Living with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicine. It's very important to take your antidepressants as prescribed, even if you start to feel better. If you stop taking them too soon, your depression could return. Stopping suddenly can also increase the risk of withdrawal side effects. Talk to a doctor or pharmacist if you have any questions or concerns about the medicine you're taking. The leaflet that comes with your medicine will have information about possible interactions with other medicines or supplements. Check with a doctor first if you plan to take any over-the-counter remedies such as painkillers, or any nutritional supplements. These can sometimes interfere with an...",9.89,11.62
2,Are there different types of depressions?,"Context: a reason and pass on their own. But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months. I learnt to cope by telling myself I don't care. Telling myself emphatically that nothing was important, I just don’t care, was my way of coping Read David's story Are there different types of depression? If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're l

In [15]:
result_df = run_experiment(QUESTIONS, TEMPLATES['complex'], is_baseline=False)

Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb

In [16]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nWhat is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to...","Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nOverview - Depression in adults. Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can ""snap out of"" by ""pulling yourself together"". The good news is that with the right treatment and support, most ...",10.41,11.67
1,What lifestyle changes are suggested for depression?,"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nmake ourselves feel better or distract ourselves. But they can make us feel worse in the long term. Sleep, diet and exercise. Some of us may struggle with sleeping, getting active, or keeping a healthy diet. And if we find these things difficult, it can affect our mood. These things are unlikely to cause depression on their own. But they could make us more vulnerable to it. for other activities you enjoy, such as: Gaming. Reading. Watching films. For more ideas, see our pages on arts and creative therapies, nature and mental health, sleep problems, st...","Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nLiving with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicine. It's very important to take your antidepressants as prescribed, even if you start to feel better. If you stop taking them too soon, your depression could return. Stopping suddenly can also increase the risk of withdrawal side effects. Talk to a doctor or pharmacist if you have any questions or concerns about the medicine you're taking. The leaflet that comes with your medicine will have information ab...",10.14,11.83
2,Are there different types of depressions?,"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\na reason and pass on their own. But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months. I learnt to cope by telling myself I don't care. Telling myself emphatically that nothing was important, I just don’t care, was my way of coping Read David's

In [17]:
result_df = run_experiment(QUESTIONS, TEMPLATES['advanced'], is_baseline=False)

Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and les

In [18]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\nWhat is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how muc...","You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\nOverview - Depression in adults. Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial...",10.62,12.13
1,What lifestyle changes are suggested for depression?,"You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\nmake ourselves feel better or distract ourselves. But they can make us feel worse in the long term. Sleep, diet and exercise. Some of us may struggle with sleeping, getting active, or keeping a healthy diet. And if we find these things difficult, it can affect our mood. These things are unlike...","You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\nLiving with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicine. It's very important to take your antidepressants as prescribed, even if you start to feel better. If you stop taking them too soon, your depr...",10.18,11.58
2,Are there different types of depressions?,"You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\na reason and pass on their own. But it might be depression if the feeling

### RedPajama 3B

Huggingface: https://huggingface.co/togethercomputer/RedPajama-INCITE-Instruct-3B-v1

> Note: RedPajama being a instruct tuned model performs worse. It also behaves like a text completion model and not a instruction-tuned model.

In [5]:
result_df = run_experiment(QUESTIONS, TEMPLATES['simple'], is_baseline=False)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood become depression? We all have times when our mood is low, and we feel sad or fed up. Often these feelings happen for a reason and pass on their own. But it might be depression if the feelings be

In [6]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings. It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood become d...","Context: Overview - Depression in adults. Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can ""snap out of"" by ""pulling yourself together"". The good news is that with the right treatment and support, most people with depression can make a full recovery. These pages are about depression in adults. Read about depression in children and young people. How to tell if you have depression. Depression affects people in different ways and ...",18.18,15.52
1,What lifestyle changes are suggested for depression?,"Context: make ourselves feel better or distract ourselves. But they can make us feel worse in the long term. Sleep, diet and exercise. Some of us may struggle with sleeping, getting active, or keeping a healthy diet. And if we find these things difficult, it can affect our mood. These things are unlikely to cause depression on their own. But they could make us more vulnerable to it. for other activities you enjoy, such as: Gaming. Reading. Watching films. For more ideas, see our pages on arts and creative therapies, nature and mental health, sleep problems, stress, relaxation and mindfulness. Think about your diet. It can be difficult to maintain a healthy diet when we’re struggling with our mental health. But eating regularly can keep your blood sugar stable and really make a differe...","Context: Living with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicine. It's very important to take your antidepressants as prescribed, even if you start to feel better. If you stop taking them too soon, your depression could return. Stopping suddenly can also increase the risk of withdrawal side effects. Talk to a doctor or pharmacist if you have any questions or concerns about the medicine you're taking. The leaflet that comes with your medicine will have information about possible interactions with other medicines or supplements. Check with a doctor first if you plan to take any over-the-counter remedies such as painkillers, or any nutritional supplements. These can sometimes interfere with an...",13.99,15.37
2,Are there different types of depressions?,"Context: a reason and pass on their own. But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months. I learnt to cope by telling myself I don't care. Telling myself emphatically that nothing was important, I just don’t care, was my way of coping Read David's story Are there different types of depression? If you're diagnosed with depression, you might also be told that it is ‘less severe’ or ‘more severe’. This describes how your symptoms are affecting you, and what treatment you're 

In [7]:
result_df = run_experiment(QUESTIONS, TEMPLATES['complex'], is_baseline=False)

Prompt to LLM : Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb

In [8]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nWhat is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to...","Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nOverview - Depression in adults. Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial and not a genuine health condition. They're wrong – it is a real illness with real symptoms. Depression is not a sign of weakness or something you can ""snap out of"" by ""pulling yourself together"". The good news is that with the right treatment and support, most ...",14.45,15.59
1,What lifestyle changes are suggested for depression?,"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nmake ourselves feel better or distract ourselves. But they can make us feel worse in the long term. Sleep, diet and exercise. Some of us may struggle with sleeping, getting active, or keeping a healthy diet. And if we find these things difficult, it can affect our mood. These things are unlikely to cause depression on their own. But they could make us more vulnerable to it. for other activities you enjoy, such as: Gaming. Reading. Watching films. For more ideas, see our pages on arts and creative therapies, nature and mental health, sleep problems, st...","Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nLiving with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicine. It's very important to take your antidepressants as prescribed, even if you start to feel better. If you stop taking them too soon, your depression could return. Stopping suddenly can also increase the risk of withdrawal side effects. Talk to a doctor or pharmacist if you have any questions or concerns about the medicine you're taking. The leaflet that comes with your medicine will have information ab...",14.07,15.49
2,Are there different types of depressions?,"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\na reason and pass on their own. But it might be depression if the feelings become so bad that they interfere with our daily life. Or if they last for several weeks or months. I learnt to cope by telling myself I don't care. Telling myself emphatically that nothing was important, I just don’t care, was my way of coping Read David's

In [9]:
result_df = run_experiment(QUESTIONS, TEMPLATES['advanced'], is_baseline=False)

Prompt to LLM : You are a highly skilled AI trained in language comprehension and summarisation.
I would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.
Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.
Please avoid unnecessary details or tangential points.
What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and les

In [10]:
result_df

,questions,mind_responses,nhs_responses,mind_time,nhs_time
0,Can you tell me about depression?,"You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\nWhat is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how muc...","You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\nOverview - Depression in adults. Depression is more than simply feeling unhappy or fed up for a few days. Most people go through periods of feeling down, but when you're depressed you feel persistently sad for weeks or months, rather than just a few days. Some people think depression is trivial...",14.51,15.68
1,What lifestyle changes are suggested for depression?,"You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\nmake ourselves feel better or distract ourselves. But they can make us feel worse in the long term. Sleep, diet and exercise. Some of us may struggle with sleeping, getting active, or keeping a healthy diet. And if we find these things difficult, it can affect our mood. These things are unlike...","You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\nLiving with - Depression in adults. There are some key steps you can take to lift your mood and help your recovery from depression. Taking your medicine. It's very important to take your antidepressants as prescribed, even if you start to feel better. If you stop taking them too soon, your depr...",14.12,15.52
2,Are there different types of depressions?,"You are a highly skilled AI trained in language comprehension and summarisation.\nI would like you to read the following text and summarise it into a concise abstract paragraph. Use the following pieces of context to answer the question at the end.\nAim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text.\nPlease avoid unnecessary details or tangential points.\na reason and pass on their own. But it might be depression if the feeling

In [ ]:
result_df = run_experiment(QUESTIONS, TEMPLATES['simple'], is_baseline=False)

In [ ]:
result_df

### Flan-T5 Large

In [5]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

Prompt to LLM : Context: What is depression? Depression is a mental health problem that involves having a low mood or losing interest and enjoyment in things. It can also cause a range of other changes to how you feel or behave. The symptoms you experience may vary. How intense they are, how long they last, and how much they affect your daily life can also vary. If you experience milder depression, you might have low mood but still be able to carry on with your daily life. But things may feel harder and less worthwhile. If you have more severe depression, you might find day-to-day life much more difficult. You may also experience suicidal feelings . It starts as sadness then I feel myself shutting down, becoming less capable of coping. Eventually, I just feel numb and empty. When does low mood difficult experience. It may also feel hard to explain your thoughts and feelings to others. You might want to cut yourself off from other people instead. This could lead to you feeling isolated 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/dudeperf3ct/.cache/pypoetry/virtualenvs/mindgpt-N4bnLkub-py3.10/lib/python3.10/site-packag │
│ es/urllib3/connectionpool.py:714 in urlopen                                                      │
│                                                                                                  │
│    711 │   │   │   │   self._prepare_proxy(conn)                                                 │
│    712 │   │   │                                                                                 │
│    713 │   │   │   # Make the request on the httplib connection object.                          │
│ ❱  714 │   │   │   httplib_response = self._make_request(                                        │
│    715 │   │   │   │   conn,                                                                     │
│    716 │   │   │   │   method,                                                                   │
│    717 │   │   │   │   url,                                                                      │
│                                                                                                  │
│ /home/dudeperf3ct/.cache/pypoetry/virtualenvs/mindgpt-N4bnLkub-py3.10/lib/python3.10/site-packag │
│ es/urllib3/connectionpool.py:466 in _make_request                                                │
│                                                                                                  │
│    463 │   │   │   │   │   # Remove the TypeError from the exception chain in                    │
│    464 │   │   │   │   │   # Python 3 (including for exceptions like SystemExit).                │
│    465 │   │   │   │   │   # Otherwise it looks like a bug in the code.                          │
│ ❱  466 │   │   │   │   │   six.raise_from(e, None)                                               │
│    467 │   │   except (SocketTimeout, BaseSSLError, SocketError) as e:                           │
│    468 │   │   │   self._raise_timeout(err=e, url=url, timeout_value=read_timeout)               │
│    469 │   │   │   raise                                                                         │
│ <string>:3 in raise_from                                                                         │
│                                                                                                  │
│ /home/dudeperf3ct/.cache/pypoetry/virtualenvs/mindgpt-N4bnLkub-py3.10/lib/python3.10/site-packag │
│ es/urllib3/connectionpool.py:461 in _make_request                                                │
│                                                                                                  │
│    458 │   │   │   except TypeError:                                                             │
│    459 │   │   │   │   # Python 3                                                                │
│    460 │   │   │   │   try:                                                                      │
│ ❱  461 │   │   │   │   │   httplib_response = conn.getresponse()                                 │
│    462 │   │   │   │   except BaseException as e:                                                │
│    463 │   │   │   │   │   # Remove the TypeError from the exception chain in                    │
│    464 │   │   │   │   │   # Python 3 (including for exceptions like SystemExit).                │
│                                                                                                  │
│ /home/dudeperf3ct/.pyenv/versions/3.10.5/lib/python3.10/http/client.py:1374 in getresponse       │
│                                                                                                  │
│   1371 │   │                                                                                     │
│   1372 │   │   try:                                                                              │
│   1373 │   │   │   try:                                    

In [ ]:
result_df

### Flan-T5 XL (OOM)

In [ ]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

In [ ]:
result_df

### Flan-T5 Large

In [ ]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

In [ ]:
result_df

### Flan-T5 XL

In [ ]:
result_df = run_experiment(QUESTIONS, is_baseline=False)

In [ ]:
result_df